In [18]:
from nltk.corpus import stopwords
import string
import re
from os import listdir
from pickle import dump

In [19]:
#loading doc into memory
def load_doc(filename):
    
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


#turning a document into a clean token
def clean_doc(doc):
    tokens = doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [20]:
filename = 'skills.txt'

In [21]:
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['econometrics', 'excelvba', 'financialanalysis', 'financialmodeling', 'financialwriting', 'forextrading', 'investmentresearch', 'python', 'statistics', 'centos', 'computerengineering', 'etl', 'ionicframework', 'machinelearning', 'datascience', 'seleniumwebdriver', 'tableau', 'dataanalysis', 'dataentry', 'databasedesign', 'databasetesting', 'java', 'microsoftaccessprogramming', 'microsoftoffice', 'microsoftsqlserveradministration', 'articlewriting', 'businessanalysis', 'businessintelligence', 'businessplans', 'businessprocessmodelling', 'businesswriting', 'contentwriting', 'creativewriting', 'microsoftexcel', 'projectmanagement', 'apachespark', 'matlab', 'stata', 'statisticalcomputing', 'bioinformatics', 'datamining', 'datavisualization', 'javascript', 'pythonnumpy', 'pythonscipy', 'scientificresearch', 'bigdata', 'predictiveanalytics', 'primavera', 'sql', 'pandas', 'hadoop', 'mongodb', 'mysql', 'artificialneuralnetworks', 'datamodeling', 'recommendersystems', 'electricalengineering', 

In [5]:
#loading all the documents in a directory
def process_docs(directory, is_train):
    documents = list()
    for filename in listdir(directory):
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
            
        path = directory + '/' + filename
        doc = load_doc(path)
        tokens = clean_doc(doc)
        documents.append(tokens)
    return documents    

In [6]:
#Loading and Cleaning the dataset
def load_clean_dataset(is_train):
    neg = process_docs('review_polarity/txt_sentoken/neg', is_train)
    pos = process_docs('review_polarity/txt_sentoken/pos', is_train)
    docs = neg + pos
    
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

In [7]:
#saving a dataset to a file
def save_dataset(dataset, filename):
    dump(dataset, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [8]:
train_docs, ytrain = load_clean_dataset(True)
test_docs, ytest = load_clean_dataset(False)

save_dataset([train_docs, ytrain], 'train.pkl')
save_dataset([test_docs, ytest], 'test.pkl')

Saved: train.pkl
Saved: test.pkl


In [9]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [10]:
# load a clean dataset
def load_dataset(filename):
    return load(open(filename, 'rb'))
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

In [11]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [12]:
# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    model.summary()
    plot_model(model, show_shapes=True, to_file='model.png')
    return model

In [13]:
# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = len(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], trainLabels, epochs=7, batch_size=16)
# save the model
model.save('model.h5')

Max document length: 1800
Vocabulary size: 44277
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1800)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1800)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1800)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1800, 100)    4427700     input_1[0][0]                    
___________________________________________

/home/aman/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.7008 - accuracy: 0.4933
Epoch 2/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.4856
Epoch 3/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.4933
Epoch 4/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 5/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 6/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.4911
Epoch 7/7
1800/1800 [==============================] - 20s 11ms/step - loss: 0.6932 - accuracy: 0.4933


In [14]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
# load a clean dataset
def load_dataset(filename):
    return load(open(filename, 'rb'))
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [16]:
trainLines, trainLabels = load_dataset('train.pkl')
testLines, testLabels = load_dataset('test.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = len(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
testX = encode_text(tokenizer, testLines, length)
# load the model
model = load_model('model.h5')
# evaluate model on training dataset
_, acc = model.evaluate([trainX,trainX,trainX], trainLabels, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset dataset
_, acc = model.evaluate([testX,testX,testX], testLabels, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

Max document length: 1800
Vocabulary size: 44277


/home/aman/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train Accuracy: 50.00
Test Accuracy: 50.00
